In [14]:
import numpy as np
import pandas as pd
import random

from tqdm import tqdm

import mne

from feature_extraction import M_bands, powervals

mne.set_log_level(40)

In [85]:
def non_seizure_retrieval(seizurelines, path,i1=None, i2=None):
    sf = 256
    s = sf * 100
    seizures = []
    df = pd.read_csv(path)[['Patient', 'File', 'Duration']]
    for file in tqdm(df.iterrows()):
        try:
            # iterates through each record with seizures (should be 142)
            path = f'../chbmit/1.0.0/{file[1][0]}/{file[1][1]}.edf'
            data = mne.io.read_raw_edf(path, preload=True)
            raw_data = data.get_data()
            if i1 != None and i2 != None:
                raw_data = raw_data[i1:i2]
            duration = file[1][2]
            start = random.randint(1, duration-1) * sf
            nonseizurevector = raw_data.take(indices=range(start-s, start+s), axis=1)
            # USE FOR DEBUGGING print(f'Length of Seizure Vector {((start + s) - (start - s)) / 256}')
            bands = M_bands(nonseizurevector, 8)
            bands = np.array(np.split(bands, 512, 2))
            bands = np.swapaxes(bands, 0, 3)
            bands = np.swapaxes(bands, 0, 2)
            bands = np.swapaxes(bands, 0, 1)
            pv = np.apply_along_axis(powervals, 3, bands)
            pv = np.swapaxes(pv, 0, 2)
            pv = np.swapaxes(pv, 0, 1)
            pv = np.hstack((pv[0:23]))
            seizures.append(pv)
        except Exception as e:
            print(e)
    try:
        nonseizures = np.array(seizures)
        return nonseizures
    except:
        return nonseizures

In [86]:
df = pd.read_csv('nonseizure.csv')[['Patient', 'File', 'Duration']]

In [87]:
with open('../chbmit/1.0.0/RECORDS', 'r') as f:
    recordlines = f.readlines()
    
with open('../chbmit/1.0.0/RECORDS-WITH-SEIZURES', 'r') as f:
    seizurelines = f.readlines()
    seizurelines = [line[6:].strip() for line in seizurelines]
    
summaries = []
for x in range(1, 25):
    if x < 10:
        x = f'0{x}'
    with open(f'../chbmit/1.0.0/chb{x}/chb{x}-summary.txt', 'r') as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines[29:]]
        summaries += lines

In [88]:
data = non_seizure_retrieval(seizurelines, 'nonseizure.csv', i1=0, i2=3)

31it [00:31,  1.01s/it]


In [90]:
data.shape

(31, 100, 24)

In [91]:
from nonseizure_dataretrieval import non_seizure_retrieval

In [92]:
data = non_seizure_retrieval(seizurelines, 'nonseizure.csv', i1=0, i2=3)

31it [00:30,  1.00it/s]
